In [1]:
import os
import cv2
import openpyxl

def create_excel_file(filename):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Student"
    sheet.append(["Name", "Class", "Face_Data", "Trained"])
    workbook.save(filename)

def create_student_dataset(student_name, student_class, dataset_path):
    student_dir = os.path.join(dataset_path, student_name)
    os.makedirs(student_dir, exist_ok=True)
    return student_dir, student_class

def capture_face_data(student_dir, student_name, student_class, webcam_id=0, num_images=80):
    # Initialize face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize video capture
    cap = cv2.VideoCapture(webcam_id)

    # Initialize sample counter
    sample_num = 0

    while sample_num < num_images:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        # Iterate over detected faces
        for (x, y, w, h) in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Save the detected face image
            face_img_path = os.path.join(student_dir, f"{student_name}_{sample_num}.png")
            cv2.imwrite(face_img_path, gray[y:y+h, x:x+w])

            # Increment sample counter
            sample_num += 1

            # Display frame
            cv2.imshow('Capturing Face Data', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

    return sample_num

def update_excel(student_name, student_class, success):
    workbook = openpyxl.load_workbook('students.xlsx')
    sheet = workbook.active

    for row in sheet.iter_rows(min_row=2, max_col=2, max_row=sheet.max_row):
        if row[0].value == student_name:
            row[1].value = student_class
            if success:
                row[2].value = "✔"
            else:
                row[2].value = "✘"
            break
    else:
        # If the student's name is not found, add a new row
        sheet.append([student_name, student_class, "✔" if success else "✘"])

    workbook.save('students.xlsx')

def main():
    # Create 'datasets' directory if it doesn't exist
    dataset_path = "datasets"
    os.makedirs(dataset_path, exist_ok=True)
    
     # Check if students.xlsx exists, if not, create it
    if not os.path.exists('students.xlsx'):
        create_excel_file("students.xlsx")
    
    # Ask for student's name and class
    student_name = input("Enter the student's name: ")
    student_class = input("Enter the student's class: ")

    # Create directory for the student's dataset
    student_dir, student_class = create_student_dataset(student_name, student_class, dataset_path)

    # Capture face data for the student
    num_images_captured = capture_face_data(student_dir, student_name, student_class)

    if num_images_captured == 80:
        update_excel(student_name, student_class, True)
        print(f"Face data captured successfully for {student_name}.")
    else:
        update_excel(student_name, student_class, False)
        print("Face data capture unsuccessful.")

if __name__ == "__main__":
    main()


Enter the student's name: narayan
Enter the student's class: 12
Face data captured successfully for narayan.
